In [4]:
import numpy as np
import pandas as pd
import sklearn
import cython
import talib
import bokeh

from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

from Functions import DataReady, KijunSen

In [5]:
data = pd.read_csv("/home/porsche/Desktop/Training_data/5minData/ATOMUSDT5min.csv")

DataReady(data)

,Open,High,Low,Close,Volume
datetime,,,,,
2024-03-11 02:00:00,13.190,13.233,13.179,13.219,10036.8
2024-03-11 02:05:00,13.219,13.276,13.217,13.243,20072.7
2024-03-11 02:10:00,13.243,13.262,13.201,13.262,8939.3
2024-03-11 02:15:00,13.262,13.264,13.203,13.205,9896.9
2024-03-11 02:20:00,13.205,13.209,13.150,13.167,8269.5
...,...,...,...,...,...
2024-04-02 19:05:00,10.943,11.030,10.931,11.013,22089.6
2024-04-02 19:10:00,11.013,11.031,10.992,11.031,16393.6
2024-04-02 19:15:00,11.031,11.031,10.950,10.976,10379.1


In [6]:
class MachineLearningStrategyV3(Strategy):
    
    # Add variables to the strategy
    atr_lenght = 14
    times_atr_tp = 30
    times_atr_sl = 20
    upper_ema_lenght = 50
    lower_ema_lenght = 200
    stoch_rsi_lenght = 14
    kijun_sen_period = 26
    kijun_sen_price_range = 2

    def init(self):
        # Indicator code
        self.atr = self.I(talib.ATR, self.data.High, self.data.Low, self.data.Close, self.atr_lenght)
        self.ema_upper = self.I(talib.EMA, self.data.Close, self.upper_ema_lenght)
        self.ema_lower = self.I(talib.EMA, self.data.Close, self.lower_ema_lenght)
        self.stoch_rsi = self.I(talib.STOCHRSI, self.data.Close, self.stoch_rsi_lenght)
        self.kijun_sen = self.I(KijunSen, data, self.kijun_sen_period)
        self.kijun_sen_to_price = self.data.Close - self.kijun_sen

        
    def next(self):
        if crossover(self.ema_lower, self.ema_upper) and self.kijun_sen_to_price < self.kijun_sen_price_range / 10:
            self.buy(sl = self.data.Close - (self.atr*self.times_atr_sl / 10), tp = self.data.Close + (self.atr*self.times_atr_tp / 10))
        

    
bt = Backtest(data, MachineLearningStrategyV3, cash = 10000, hedging = True)

stats = bt.optimize(
    random_state = 1,
    max_tries = 1000,
    return_heatmap = True,
    method = "skopt",
    upper_ema_lenght = range(20, 100, 1),
    lower_ema_lenght = range(100, 200, 1),
    kijun_sen_period = range(10, 40, 1),
    times_atr_tp = range(10, 30, 1),
    times_atr_sl = range(10, 30, 1),
    atr_lenght = range(5, 30, 1),
    stoch_rsi_lenght = range(5, 40, 1),
    kijun_sen_price_range = range(0, 10, 1),
    maximize = "Sharpe Ratio"
)

print(stats)

bt.plot()

(Start                     2024-03-11 02:00:00
End                       2024-04-02 19:25:00
Duration                     22 days 17:25:00
Exposure Time [%]                    0.596878
Equity Final [$]                  1164.415011
Equity Peak [$]                   1198.593999
Return [%]                          16.441501
Buy & Hold Return [%]              -17.172252
Return (Ann.) [%]                 1019.700761
Volatility (Ann.) [%]              370.267624
Sharpe Ratio                         2.753956
Sortino Ratio                       78.635478
Calmar Ratio                       351.854376
Max. Drawdown [%]                   -2.898076
Avg. Drawdown [%]                   -2.223628
Max. Drawdown Duration        4 days 04:35:00
Avg. Drawdown Duration        1 days 17:48:00
# Trades                                   17
Win Rate [%]                        64.705882
Best Trade [%]                       0.727435
Worst Trade [%]                     -0.426227
Avg. Trade [%]                   

/home/porsche/Desktop/Turbo-Bot/.venv/lib/python3.12/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/home/porsche/Desktop/Turbo-Bot/.venv/lib/python3.12/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/home/porsche/Desktop/Turbo-Bot/.venv/lib/python3.12/site-packages/backtesting/_plotting.py:456: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  .resample(resample_rule, label='left')


GridPlot(id='p3082', ...)